In [2]:
import json
import os
from datetime import date
from medcat.cat import CAT
from medcat.meta_cat import MetaCAT
from medcat.config_meta_cat import ConfigMetaCAT
from medcat.tokenizers.meta_cat_tokenizers import TokenizerWrapperBERT

/Users/k1897038/projects/MedCAT/medcat/cat.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# if you want to enable info level logging
import logging
logging.basicConfig(level=logging.INFO,force=True)

In [3]:
ner_model = '/Users/k1897038/Documents/cogstack_docs/medcat_models/Snomed2024-10_GSTT_KCH_2025-01-06_metacat_v1_4920d37f9bd575c4.zip'
cat = CAT.load_model_pack(ner_model)

/Users/k1897038/miniconda3/envs/cattrainer/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Load the model pack with MetaCATs


In [3]:
model_pack = '<enter model pack location>' # .zip model pack location 
mctrainer_export = "<enter mct export location>"  # name of your mct export


In [ ]:
# Load model
cat = CAT.load_model_pack(model_pack)

In [4]:

# Check what meta cat models are in this model pack.
print(f'There are: {len(cat._meta_cats)} meta cat models in this model pack.')


There are: 3 meta cat models in this model pack.


In [6]:
print(cat._meta_cats[0])

{
  "Category Name": "Presence",
  "Description": "No description",
  "Classes": {
    "False": 2,
    "Hypothetical": 1,
    "True": 0
  },
  "Model": "lstm"
}


In [7]:
print(cat._meta_cats[1])

{
  "Category Name": "Subject",
  "Description": "No description",
  "Classes": {
    "Other": 0,
    "Patient": 1
  },
  "Model": "lstm"
}


In [8]:
print(cat._meta_cats[2])

{
  "Category Name": "Time",
  "Description": "No description",
  "Classes": {
    "Future": 0,
    "Past": 2,
    "Recent": 1
  },
  "Model": "lstm"
}


<b> NOTE: </b> 
 The name for the classification task can vary. E.g: The Category Name for 'Experiencer' can be 'Subject', as it has been configured an annoated in MedCATTrainer this way, but the model expects 'Experiencer'
 
 To accomodate for this, we have a list that stores the variations for the alternate names. This attribute can be found under `mc.config.general.alternative_category_names`

E.g. for Experiencer, it will be pre-loaded as alternative_category_names = ['Experiencer','Subject']

Set this list to ensure during training / fine-tuning the model is aware of alternative names for classes.

In [13]:
cat._meta_cats[0].config.general.alternative_category_names

[]

<b> NOTE: </b> 
 The name for the classes can vary too. Some sites may have trained a MetaCAT model for the same task, but called a class value a slightly different name.
 
 E.g: For the Presence task, the class name can be 'Not present (False)' or 'False'
 
 To accomodate for this, we have a mapping that stores the variations for the alternate names. This attribute can be found under `mc.config.general.alternative_class_names`

 E.g. for Presence, it will be pre-loaded as alternative_class_names = [["Hypothetical (N/A)","Hypothetical"],["Not present (False)","False"],["Present (True)","True"]]

# For LSTM and BERT model

In [ ]:
# Train the first meta cat model - 'Prescence' Task.
meta_cat = cat._meta_cats[0]
# use this to map tasks that are equivalent. For example:
meta_cat.config.general.alternative_category_names = ['Prescence', 'Negated']

# use this to map classes that are equivalent. For example:
meta_cat.config.general.alternative_class_names = [["Hypothetical (N/A)","Hypothetical"],["Not present (False)","False"],["Present (True)","True"]]

# to overwrite the existing model, resave the fine-tuned model with the same model pack dir
meta_cat_task = meta_cat.config.general.category_name
save_dir_path = os.path.join('<model pack dir>',"meta_"+ meta_cat_task)
# to save the new model elsewhere, uncomment the below line
#save_dir_path= "test_meta_"+meta_model # Where to save the meta_model and results. 

# train the meta_model
results = meta_cat.train_from_json(mctrainer_export, save_dir_path=save_dir_path)

# Save results
json.dump(results['report'], open(os.path.join(save_dir_path,'meta_'+meta_cat_task+'_results.json'), 'w'))

## If you dont have the model packs, and are training from scratch
<b>This is very rare, it is recommended to always use the model packs and then fine-tune them</b>

In [ ]:
config = ConfigMetaCAT()
# make sure to change the following parameters:
# config.model['nclasses']
# config.general['category_name']

# change model name if training BERT for the first time
config.model['model_name'] = 'bert'

tokenizer = TokenizerWrapperBERT.load("", config.model['model_variant'])

save_dir_path= "test_meta_" + meta_cat_task # Where to save the meta_model and results. 

# Initialise and train meta_model
mc = MetaCAT(tokenizer=tokenizer, embeddings=None, config=config)
results = mc.train_from_json(mctrainer_export, save_dir_path=save_dir_path)

# Save results
json.dump(results['report'], open(os.path.join(save_dir_path,'meta_' + meta_cat_task+'_results.json'), 'w'))